# Text generation using RNN/LSTM
This code implements a Recurrent Neural Network with LSTM/RNN units for training/sampling from character-level language models. In other words, the model takes a text file as input and trains the RNN network that learns to predict the next character in a sequence.  
The RNN can then be used to generate text, character by character that will look like the original training data. 

The code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and it is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).

First, import the requiered libraries:

In [1]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np

### Data loader
The following cell is a class that help to read data from input file.

In [2]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small.
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

### Parameters
#### Batch, number_of_batch, batch_size and seq_length

Lets assume the input is this sentence: '__here is an example__'. Then:
- txt_length = 18  
- seq_length = 3  
- batch_size = 2  
- number_of_batchs = 18 / 3 * 2 = 3
- batch = array(['h', 'e', 'r'], ['e', ' ', 'i'])
- sample Seq = 'her'  

Ok, now, lets look at a real dataset, with real parameters. 

In [3]:
seq_length = 50  # RNN sequence length.
batch_size = 60  # Minibatch size, i.e. size of data in each epoch.
num_epochs = 125
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128  # Size of RNN hidden state (output dimension).
num_layers = 2  # Number of layers in the RNN.

Print part of the input file:

In [1]:
with open('input.txt', 'r') as f:
    read_data = f.read()
    print(read_data[0:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


Now, we can read the data at batches using the `TextLoader`. It will convert the characters to numbers, and represent each sequence as a vector in batches:

In [8]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print("vocabulary size:", data_loader.vocab_size)
print("Characters:", data_loader.chars)
print("vocab number of 'F':", data_loader.vocab['F'])
print("Character sequences (first batch):", data_loader.x_batches[0])

loading preprocessed files
vocabulary size: 65
Characters: (' ', 'e', 't', 'o', 'a', 'h', 's', 'r', 'n', 'i', '\n', 'l', 'd', 'u', 'm', 'y', ',', 'w', 'f', 'c', 'g', 'I', 'b', 'p', ':', '.', 'A', 'v', 'k', 'T', "'", 'E', 'O', 'N', 'R', 'S', 'L', 'C', ';', 'W', 'U', 'H', 'M', 'B', '?', 'G', '!', 'D', '-', 'F', 'Y', 'P', 'K', 'V', 'j', 'q', 'x', 'z', 'J', 'Q', 'Z', 'X', '3', '&', '$')
vocab number of 'F': 49
Character sequences (first batch): [[49  9  7 ...  1  4  7]
 [19  4 14 ... 14  9 20]
 [ 8 20 10 ...  8 10 18]
 ...
 [21  2  0 ...  0 21  0]
 [ 9  7  7 ...  0  2  3]
 [ 3  7  0 ...  5  9 23]]


### Input and output

In [9]:
x, y = data_loader.next_batch()
x

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]])

In [10]:
x.shape  # batch_size = 60, seq_length = 50

(60, 50)

Here, `y` is the next character for each character in `x`:

In [11]:
y

array([[ 9,  7,  6, ...,  4,  7,  0],
       [ 4, 14, 22, ...,  9, 20,  5],
       [20, 10, 29, ..., 10, 18,  4],
       ...,
       [ 2,  0,  6, ..., 21,  0,  6],
       [ 7,  7,  4, ...,  2,  3,  0],
       [ 7,  0, 33, ...,  9, 23,  0]])

### LSTM Architecture
Each LSTM cell has 5 parts:
1. Input
2. prv_state
3. prv_output
4. new_state
5. new_output


- Each LSTM cell has an input layer, which size is 128 units in our case. The input vector's dimension also is 128, which is the dimensionality of embedding vector, so called, dimension size of W2V/embedding, for each character/word.
- Each LSTM cell has a hidden layer, where there are some hidden units. The argument `n_hidden = 128` of BasicLSTMCell is the number of hidden units of the LSTM (inside A). It keeps the size of the output and state vector. It is also known as, rnn_size, num_units, num_hidden_units, and LSTM size
- An LSTM keeps two pieces of information as it propagates through time: 
    - __hidden state__ vector: Each LSTM cell accepts a vector, called __hidden state__ vector, of size `n_hidden = 128`, and its value is returned to the LSTM cell in the next step. The __hidden state__ vector which is the memory of the LSTM, accumulates using its (forget, input and output) gates through time. "num_units" is equivalant to "size of RNN hidden state". number of hidden units is the dimensianality of the output (dimesianality of the state) of the LSTM cell.
    - __previous time-step output__: For each LSTM cell that we initialize, we need to supply a value (128 in this case) for the hidden dimension, or as some people like to call it, the number of units in the LSTM cell. 


`num_layers = 2`  
- Number of layers in the RNN, is defined by num_layers
- An input of `MultiRNNCell` is `cells` which is a list of RNNCells that will be composed in this order.

### Defining stacked RNN Cell

`BasicRNNCell` is the most basic RNN cell.

In [12]:
cell = tf.contrib.rnn.BasicRNNCell(rnn_size)

In [13]:
# A two layer cell.
stacked_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)

In [14]:
# Hidden state size.
stacked_cell.output_size

128

`state` varibale keeps output and new_state of the LSTM, so it is a tuple of size:

In [15]:
stacked_cell.state_size

(128, 128)

Lets define input data:

In [16]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
input_data

<tf.Tensor 'Placeholder:0' shape=(60, 50) dtype=int32>

And target data:

In [17]:
targets = tf.placeholder(tf.int32, [batch_size, seq_length])
targets

<tf.Tensor 'Placeholder_1:0' shape=(60, 50) dtype=int32>

The memory state of the network is initialized with a vector of zeros and gets updated after reading each character.

`BasicRNNCell.zero_state(batch_size, dtype)` returns zero-filled state tensor(s). In this function, batch_size
representing the batch size.

In [18]:
initial_state = stacked_cell.zero_state(batch_size, tf.float32)

Lets check the value of the input_data again:

In [19]:
session = tf.Session()
feed_dict = {input_data: x, targets: y}
session.run(input_data, feed_dict)

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]], dtype=int32)

### Embedding
In this section, we build a 128-dim vector for each character. As we have 60 batches, and 50 characters in each sequence, it will generate a [60, 50, 128] matrix.

__Notice:__ The function `tf.get_variable()` is used to share a variable and to initialize it in one place. `tf.get_variable()` is used to get or create a variable instead of a direct call to `tf.Variable`. 

In [20]:
with tf.variable_scope('rnnlm', reuse=False):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])  # 128x65
    softmax_b = tf.get_variable("softmax_b", [vocab_size])  # 1x65
        
    # Embedding variable is initialized randomly.
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  # 65x128

    # embedding_lookup goes to each row of input_data, and for each character in the row,
    # finds the correspond vector in embedding.
    # Creates a 60*50*[1*128] matrix.
    # So, the first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character.
    em = tf.nn.embedding_lookup(embedding, input_data)  # em is 60x50x[1*128]
    # split: Splits a tensor into sub tensors.
    # It will split the 60x50x[1x128] matrix into 50 matrix of 60x[1*128]
    inputs = tf.split(em, seq_length, 1)
    # Convert the list to 50 matrix of [60x128]
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

Lets take a look at the __embedding__, __em__ and __inputs__ variables:

Embedding variable is initialized with random values:

In [21]:
session.run(tf.global_variables_initializer())
session.run(embedding)

array([[ 0.1722724 , -0.08362447, -0.12620343, ..., -0.09066197,
        -0.10572398,  0.03630777],
       [ 0.0302359 , -0.16806553, -0.10838205, ...,  0.16656597,
         0.11187743, -0.04912362],
       [ 0.07656111,  0.07549156,  0.06535269, ..., -0.02334714,
        -0.03505114, -0.075584  ],
       ...,
       [-0.15861222, -0.04039523, -0.00890502, ..., -0.1521869 ,
        -0.14659537,  0.06569719],
       [-0.15520895,  0.11477567, -0.14152746, ...,  0.11538155,
         0.05942373, -0.03053328],
       [-0.0547492 ,  0.15497552, -0.04752883, ...,  0.07466076,
         0.10189132, -0.10777494]], dtype=float32)

The first element of em, is a matrix of 50x128, which each row of it is a vector representing that character.

In [23]:
em = tf.nn.embedding_lookup(embedding, input_data)
emp = session.run(em, feed_dict={input_data: x})
print(emp.shape)
emp[0]

(60, 50, 128)


array([[ 0.08352269,  0.10425128,  0.13842289, ..., -0.12345897,
        -0.06512738, -0.07250892],
       [-0.02447218,  0.08035068, -0.04622024, ...,  0.14011173,
         0.03743592,  0.11483796],
       [ 0.02744865,  0.14238094, -0.10165575, ..., -0.09894219,
         0.12508132, -0.14850871],
       ...,
       [ 0.0302359 , -0.16806553, -0.10838205, ...,  0.16656597,
         0.11187743, -0.04912362],
       [-0.14158408, -0.08662911,  0.05569063, ...,  0.06909324,
        -0.04474066, -0.14188258],
       [ 0.02744865,  0.14238094, -0.10165575, ..., -0.09894219,
         0.12508132, -0.14850871]], dtype=float32)

Let's consider each sequence as a sentence of length 50 characters, then, the first item in __inputs__ is a [60x128] vector which represents the first characters of 60 sentences.

In [24]:
inputs = tf.split(em, seq_length, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
inputs[0:5]

[<tf.Tensor 'Squeeze:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_2:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_4:0' shape=(60, 128) dtype=float32>]

### Feeding a batch of 50 sequences to a RNN:

The feeding process for inputs is as following:

- Step 1:  first character of each of the 50 sentences (in a batch) is entered in parallel.  
- Step 2:  second character of each of the 50 sentences is input in parallel. 
- Step n: nth character of each of the 50 sentences is input in parallel.  

The parallelism is only for efficiency.  Each character in a batch is handled in parallel,  but the network sees one character of a sequence at a time and does the computations accordingly. All the computations involving the characters of all sequences in a batch at a given time step are done in parallel. 

In [25]:
session.run(inputs[0], feed_dict={input_data: x})

array([[ 0.08352269,  0.10425128,  0.13842289, ..., -0.12345897,
        -0.06512738, -0.07250892],
       [-0.037874  , -0.07817087,  0.1164002 , ...,  0.07249571,
        -0.0046092 ,  0.01694898],
       [ 0.03743163, -0.02072446, -0.12352531, ...,  0.11484666,
         0.13091497,  0.02999784],
       ...,
       [ 0.11660947, -0.1103497 ,  0.09357838, ...,  0.0761029 ,
        -0.03518528,  0.07144852],
       [-0.02447218,  0.08035068, -0.04622024, ...,  0.14011173,
         0.03743592,  0.11483796],
       [-0.03696737,  0.14969508, -0.12469925, ..., -0.168251  ,
         0.14400227, -0.07804833]], dtype=float32)

Feeding the RNN with one batch, we can check the new output and new state of network:

In [26]:
# outputs is 50x[60*128]
outputs, new_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, stacked_cell, loop_function=None, scope='rnnlm')
new_state

(<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_98:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_99:0' shape=(60, 128) dtype=float32>)

In [27]:
outputs[0:5]

[<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_5:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_7:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_9:0' shape=(60, 128) dtype=float32>]

Let's check the output of the network after feeding it with first batch:

In [28]:
first_output = outputs[0]
session.run(tf.global_variables_initializer())
session.run(first_output, feed_dict={input_data: x})

array([[-0.07270692,  0.00914923,  0.07391221, ...,  0.00625553,
        -0.10549624, -0.11863969],
       [-0.06421256,  0.13520257,  0.01071523, ...,  0.03852994,
         0.10306417, -0.03523347],
       [-0.13965774,  0.04955263, -0.03708952, ...,  0.00821678,
        -0.1010602 , -0.1300817 ],
       ...,
       [ 0.15044728, -0.03523955, -0.02970305, ..., -0.00524335,
        -0.14697067,  0.02718503],
       [-0.0813082 , -0.0428643 ,  0.04957348, ..., -0.09574681,
        -0.14821191,  0.10273302],
       [ 0.05410901,  0.0038401 ,  0.05570202, ..., -0.01997526,
         0.0489104 ,  0.1196968 ]], dtype=float32)

As it was explained, __outputs__ variable is a 50x[60x128] tensor. We need to reshape it back to [60x50x128] to be able to calculate the probablity of the next character using the softmax. The `softmax_w` shape is [rnn_size, vocab_size], which is [128x65] in our case. Threfore, we have a fully connected layer on top of LSTM cells, which help us to decode the next charachter. We can use the `softmax(output * softmax_w + softmax_b)` for this purpose. The shape of the matrices would be:

softmax([60x50x128]x[128x65]+[1x65]) = [60x50x65]

We can do it step-by-step:

In [29]:
output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
output

<tf.Tensor 'Reshape:0' shape=(3000, 128) dtype=float32>

In [30]:
logits = tf.matmul(output, softmax_w) + softmax_b
logits

<tf.Tensor 'add:0' shape=(3000, 65) dtype=float32>

In [31]:
probs = tf.nn.softmax(logits)
probs

<tf.Tensor 'Softmax:0' shape=(3000, 65) dtype=float32>

Here is the probablity of the next chracter in all batches:

In [32]:
session.run(tf.global_variables_initializer())
session.run(probs, feed_dict={input_data: x})

array([[0.01596195, 0.01497225, 0.01251268, ..., 0.0152396 , 0.01854368,
        0.01418732],
       [0.0152303 , 0.01380554, 0.01368246, ..., 0.01589741, 0.01796742,
        0.01024323],
       [0.0144489 , 0.01359341, 0.01239462, ..., 0.01483944, 0.01449357,
        0.01601436],
       ...,
       [0.0166013 , 0.01857689, 0.01334734, ..., 0.01840278, 0.01606724,
        0.01974845],
       [0.01209294, 0.01238278, 0.01320278, ..., 0.01759913, 0.01553547,
        0.01237708],
       [0.01651255, 0.01159354, 0.01127395, ..., 0.01653617, 0.01862071,
        0.0123069 ]], dtype=float32)

Now, we are in the position to calculate the cost of training with __loss function__, and keep feedng the network to learn it. But, the question is: what the LSTM networks learn?

In [33]:
grad_clip = 5.
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'rnnlm/softmax_w:0' shape=(128, 65) dtype=float32_ref>,
 <tf.Variable 'rnnlm/softmax_b:0' shape=(65,) dtype=float32_ref>,
 <tf.Variable 'rnnlm/embedding:0' shape=(65, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>]

# All together
Now, let's put all of parts together in a class, and train the model:

In [34]:
class LSTMModel():
    def __init__(self, sample=False):
        rnn_size = 128  # Size of RNN hidden state vector.
        batch_size = 60  # Minibatch size, i.e. size of dataset in each epoch.
        seq_length = 50  # RNN sequence length.
        num_layers = 2  # Number of layers in the RNN.
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of a LSTM cell that processes one char at a time and
        # computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated
        # after reading each char. 
        self.initial_state = stacked_cell.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])  # 128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size])  # 1x65
            with tf.device("/cpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  # 65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    def sample(self, sess, chars, vocab, num=200, prime='The ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

### Creating the LSTM object
Now we create a LSTM model:

In [35]:
with tf.variable_scope("rnn"):
    model = LSTMModel()

# Train using LSTMModel
We can train our model through feeding batches:

In [37]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs):
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state)  # (2x[60x128])
        for b in range(data_loader.num_batches):
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state: state}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
        print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, train_loss, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print(sample_model.sample(sess, data_loader.chars, data_loader.vocab, num=50, prime='The ', sampling_type=1))
            print('----------------------------------')

370/46375 (epoch 0), train_loss = 1.912, time/batch = 0.050
>> sample mode:
The sagrenous a man; aid to Advarge our lanjacnant one
----------------------------------
741/46375 (epoch 1), train_loss = 1.750, time/batch = 0.060
>> sample mode:
The within, I was head, offy dring, ane mers thill of 
----------------------------------
1112/46375 (epoch 2), train_loss = 1.677, time/batch = 0.047
>> sample mode:
The pray of throous oftentiou, you alouth belike the w
----------------------------------
1483/46375 (epoch 3), train_loss = 1.638, time/batch = 0.062
>> sample mode:
The her
First from a slears;
With; theu, late among fa
----------------------------------
1854/46375 (epoch 4), train_loss = 1.612, time/batch = 0.049
>> sample mode:
The so not he shamest you send and ever hold you, but 
----------------------------------
2225/46375 (epoch 5), train_loss = 1.596, time/batch = 0.046
>> sample mode:
The knots you stose beggamen;
And thou hand unrear hil
----------------------------------
